## Proyecto 4

Ejecutar el siguiente código para asegurar que las librerías están cargadas

In [3]:
modules = ["yfinance"]

import sys
import os.path
from subprocess import check_call
import importlib
import os

def instala(modules):
    print("Instalando módulos")
    for m in modules:
        # para el import quitamos [...] y ==...
        p = m.find("[")
        mi = m if p==-1 else m[:p]
        p = mi.find("==")
        mi = mi if p==-1 else mi[:p]
        torch_loader = importlib.util.find_spec(mi)
        if torch_loader is not None:
            print(m," encontrado")
        else:
            print(m," No encontrado, instalando...",end="")  
            try:        
                r = check_call([sys.executable, "-m", "pip", "install", "--user", m])
                print("¡hecho!")
            except:
                print("¡Problema al instalar ",m,"! ¿seguro que el módulo existe?",sep="")

    print("¡Terminado!")

instala(modules) 

Instalando módulos
yfinance  No encontrado, instalando...¡hecho!
¡Terminado!


Preparamos los datos

In [9]:

import yfinance as yf
import pandas as pd

# Definir los símbolos de los 5 primeros valores del IBEX 35
symbols = ['SAN.MC', 'BBVA.MC', 'IBE.MC', 'ITX.MC', 'REP.MC']  # cód. Yahoo
nombre = ['Santander', 'BBVA', 'Iberdrola', 'Inditex', 'Repsol']  # Nombre

# descargamos y nos quedamos solo con los valores al cierre
df_raw = yf.download(tickers=symbols, period='max').loc[:,"Adj Close"]

# solo queremos los incrementos diarios y solo filas in nulos
df_raw = df_raw.dropna().pct_change().dropna()

# renombrar columna
df_raw.columns = [nombre[symbols.index(c)]+"_hoy" for c in df_raw.columns]
# día de la semana
df_raw['Día'] = pd.to_datetime(df_raw.index).day_name(locale='es_ES') 
# semana del año
df_raw['Semana'] = df_raw.index.isocalendar().week

df_raw['BBVA_ayer'] = df_raw['BBVA_hoy'].shift(1)
df_raw['Iberdrola_ayer'] = df_raw['Iberdrola_hoy'].shift(1)
df_raw['Inditex_ayer'] = df_raw['Inditex_hoy'].shift(1)
df_raw['Repsol_ayer'] = df_raw['Repsol_hoy'].shift(1)
df_raw['Santander_ayer'] = df_raw['Santander_hoy'].shift(1)
df_raw['Santander_mañana'] = df_raw['Santander_hoy'].shift(-1)

df = df_raw.dropna()
df

[*********************100%***********************]  5 of 5 completed


,BBVA_hoy,Iberdrola_hoy,Inditex_hoy,Repsol_hoy,Santander_hoy,Día,Semana,BBVA_ayer,Iberdrola_ayer,Inditex_ayer,Repsol_ayer,Santander_ayer,Santander_mañana
Date,,,,,,,,,,,,,
2001-05-28 00:00:00+00:00,0.013333,0.000000,-0.006107,-0.005543,0.007278,Lunes,22,-0.018691,-0.022828,-0.001663,-0.006881,-0.016114,-0.009936
2001-05-29 00:00:00+00:00,0.002507,-0.009734,0.009497,-0.015327,-0.009936,Martes,22,0.013333,0.000000,-0.006107,-0.005543,0.007278,0.001825
2001-05-30 00:00:00+00:00,0.000625,-0.005242,0.001661,-0.004245,0.001825,Miércoles,22,0.002507,-0.009734,0.009497,-0.015327,-0.009936,0.024590
2001-05-31 00:00:00+00:00,0.005622,0.007905,0.013812,-0.002369,0.024590,Jueves,22,0.000625,-0.005242,0.001661,-0.004245,0.001825,-0.020444
2001-06-01 00:00:00+00:00,-0.007453,-0.009804,-0.002725,0.004274,-0.020444,Viernes,22,0.005622,0.007905,0.013812,-0.002369,0.024590,0.020871
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-03 00:00:00+00:00,0.008645,0.003666,-0.001154,0.012944,0.000226,Jueves,40,-0.004946,-0.019410,-0.006878,0.002931,0.011426,0.011972
2024-10-04 00:00:00+00:00,0.023356,-0.008766,-0.001156,0.014427,0.011972,Viernes,40,0.008645,0.003666,-0.001154,0.012944,0.000226,0.017857
2024-10-07 00:00:00+00:00,0.010469,-0.004790,0.010798,0.008939,0.017857,Lunes,41,0.023356,-0.008766,-0.001156,0.014427,0.011972,0.000439


Cada fila representa un día. Tiene valores de cierre de hoy (hoy --> fecha) de Iberdrola, Inditex, BBVA y Santander, los valores que tuvieron ayer, y el BBVA de mañana, que es la etiqueta que queremos predecir.

a) Se pide hacer un modelo de regresión lineal para predecir `BBVA_mañana`. Se puede probar con varias combinaciones de columnas para intentar predecir `BBVA_mañana`...salvo el propio `BBVA_mañana` claro.

No hay que hacer demasiado, probar con 2 o 3 combinaciones y dar el margen del error. 

b) Probar a hacer alguna predicción con el mejor modelo del apartado a; inventarse algunos valores para las columnas seleccionadas y ver qué valor predice.